In [2]:
%cd ..

/root/mll-mlops-model-pipeline-template


In [3]:
import os
import sys
import yaml
import boto3
import logging
import argparse
import sagemaker

#sys.path.append(os.path.join(os.path.dirname(__file__), ".."))
from src.model_build.pipeline import get_pipeline

logging.basicConfig(level=logging.INFO)

#if __name__ == "__main__":
parser = argparse.ArgumentParser("Creates or updates and runs the pipeline for the pipeline script.")
parser.add_argument("--run-execution", action="store_true")
args, _ = parser.parse_known_args()

# IAM ROLE
iam_role = sagemaker.get_execution_role()

# custom diaz code
# Get session info
region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session(default_bucket='mll-tfm-v3-model-pipeline-development-961105418118')
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

# set branch
os.environ["TF_VAR_branch"] = "development"

In [4]:
# CONFIG
with open("cfg/model_build.yaml") as f: config = yaml.load(f, Loader=yaml.SafeLoader)

In [5]:
try:
    # INSTANTIATE PIPELINE
    logging.info("INSTANTIATE PIPELINE")
    pipeline = get_pipeline(iam_role=iam_role, session=sagemaker_session, cfg=config)

    # CREATE/UPDATE PIPELINE IN SAGEMAKER
    logging.info("CREATE/UPDATE PIPELINE IN SAGEMAKER")

    upsert_response = pipeline.upsert(role_arn=iam_role)
    logging.info(f"SAGEMAKER PIPELINE RESPONSE RECEIVED:\n {upsert_response}")

    #if args.run_execution:
    # RUN PIPELINE
    logging.info("RUNNING PIPELINE")
    execution = pipeline.start()
    logging.info(f"EXECUTION STARTED WITH PipelineExecutionArn: {execution.arn}")

    logging.info("WAITING FOR THE EXECUTION TO FINISH...")
    execution.wait(60)

    logging.info(f"EXECUTION COMPLETED. SEE THE EXECUTION STEP DETAILS:\n {execution.list_steps()}")
except Exception as e:
    print(f"Exception: {e}")
    sys.exit(1)

INFO:root:INSTANTIATE PIPELINE
/opt/conda/lib/python3.8/site-packages/sagemaker/workflow/pipeline_context.py:233: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  warnings.warn(
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:root:CREATE/UPDATE PIPELINE IN SAGEMAKER
INFO:root:SAGEMAKER PIPELINE RESPONSE RECEIVED:
 {'PipelineArn': 'arn:aws:sagemaker:eu-central-1:961105418118:pipeline/mlops-template', 'ResponseMetadata': {'RequestId': 'e8b64948-913b-4cb5-b9c5-7edb0b44ad25', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e8b64948-913b-4cb5-b9c5-7edb0b44ad25', 'content-type': 'application/x-amz-json-1.1', 'content-length': '85', 'date': 'Tue